# Read COCO from metadata.json and save masks to nifti files

The original files are generated by `pilsen_pigs_from_dicom.ipynb`

In [3]:

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
!pip install git+https://github.com/mjirik/io3d --upgrade --force-reinstall 
!pip install git+https://github.com/mjirik/imma --upgrade --force-reinstall 
    
# !pip install git+https://github.com/mjirik/io3d 
# !pip install git+https://github.com/mjirik/imma 

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/mjirik/io3d to /var/tmp/pbs.4659475.pbs-m1.metacentrum.cz/pip-req-build-t67_uydf
  Running command git clone --filter=blob:none --quiet https://github.com/mjirik/io3d /var/tmp/pbs.4659475.pbs-m1.metacentrum.cz/pip-req-build-t67_uydf
  Resolved https://github.com/mjirik/io3d to commit 13ec0021aefdf5e2dc20c0b62d5945682cdfecb2
  Preparing metadata (setup.py) ... done
  Created wheel for io3d: filename=io3d-2.10.8-py3-none-any.whl size=81321 sha256=168af6e7ad35409a5c62ece06e9af8c653344d6def59c4044eecb6de150b260f
  Stored in directory: /var/tmp/pbs.4659475.pbs-m1.metacentrum.cz/pip-ephem-wheel-cache-7lv93nv9/wheels/12/23/e0/772468d07232994f413fa2f3b9ea57674d9454973386a06b4b
Successfully built io3d
  Attempting uninstall: io3d
    Found existing installation: io3d 2.10.7
    Uninstalling io3d-2.10.7:
      Successfully uninstalled io3d-2.10.7
Defaulting to user installation because norm

In [42]:
!pip install tqdm scikit-image loguru pydicom nibabel

Defaulting to user installation because normal site-packages is not writeable


In [43]:
import tqdm
import json
import matplotlib.pyplot as plt
import numpy as np
from skimage.draw import polygon
import skimage.io
from pathlib import Path
from loguru import logger
import io3d
import io3d.image
import os



In [44]:
import getpass

username = getpass.getuser()
username

'mjirik'

In [50]:
dataset_root = Path(r"H:\biomedical\orig\pilsen_pigs_all")
dataset_root = Path(f"/storage/plzen1/home/{username}/data/biomedical/orig/pilsen_pigs_all")
dataset_root.exists()

True

In [51]:
output_path = dataset_root
output_path = Path("~/data/biomedical/processed/pilsen_pigs_all").expanduser()



# Get shapes and voxelsizes in mhd files

In [ ]:
# dataset_mhd_path = list(dataset_root.glob("transplantation_raw/**/*.mhd"))
dataset_mhd_path = list(dataset_root.glob("transplantation_nii/**/*.nii.gz"))
metadata_path = output_path / "dataset_metadata.json"
# metadata_path = "dataset_metadata.json"

len(dataset_mhd_path)
if not metadata_path.exists():
    dataset_metadata = {}
    for fn in tqdm.tqdm(dataset_mhd_path):
        # tqdm.tqdm.write(f"Reading {fn}")
        datap = io3d.read(fn)
        fn_stem = fn.name.split(".", 1)[0]
        dataset_metadata[fn_stem] = dict(filename=fn.name, shape= datap.data3d.shape, voxelsize_mm= np.asarray(datap.voxelsize_mm).astype(float).tolist())

    metadata_path.parent.mkdir(parents=True, exist_ok=True)
    with open(metadata_path, "w") as f:
        json.dump(dataset_metadata, f)

    dataset_metadata

In [ ]:
class AnnotationTOmask:
    def __init__(self, annotation_file):
        # load dataset
        print("loading annotations ...")
        dataset = json.load(open(annotation_file, "r"))
        print("annotations loaded!")

        # creating index
        print("creating index...")
        imgToAnns = {ann["image_id"]: [] for ann in dataset["annotations"]}
        anns = {ann["id"]: [] for ann in dataset["annotations"]}
        for ann in dataset["annotations"]:
            imgToAnns[ann["image_id"]] += [ann]
            anns[ann["id"]] = ann

        imgs = {im["id"]: {} for im in dataset["images"]}
        for img in dataset["images"]:
            imgs[img["id"]] = img

        cats = []
        catToImgs = []
        cats = {cat["id"]: [] for cat in dataset["categories"]}
        for cat in dataset["categories"]:
            cats[cat["id"]] = cat
        catToImgs = {cat["id"]: [] for cat in dataset["categories"]}
        for ann in dataset["annotations"]:
            catToImgs[ann["category_id"]] += [ann["image_id"]]

        print(f"index created! {len(imgs)} images, {len(anns)} annotations, {len(cats)} categories")

        # create class members
        self.anns = anns
        self.imgToAnns = imgToAnns
        self.catToImgs = catToImgs
        self.imgs = imgs
        self.cats = cats
        self.dataset = dataset

    def getAnnIds(self, imgIds, catIds):
        """
        Get ann ids for given cats from all images
        """
        imgIds = imgIds if type(imgIds) == list else [imgIds]
        catIds = catIds if type(catIds) == list else [catIds]

        if not len(imgIds) == 0:
            anns = sum(
                [self.imgToAnns[imgId] for imgId in imgIds if imgId in self.imgToAnns],
                [],
            )
        else:
            anns = self.dataset["annotations"]
        anns = (
            anns
            if len(catIds) == 0
            else [ann for ann in anns if ann["category_id"] in catIds]
        )
        ids = [ann["id"] for ann in anns]

        return ids

    def getCatIds(self, catNms=[], catIds=[]):
        """
        get integer array of cat ids for given cat names, given cat ids
        """
        if len(catNms) == len(catIds) == 0:
            cats = self.dataset["categories"]
        else:
            # print("+++++++++++++++++++++")
            cats = self.dataset["categories"]
            cats = (
                cats
                if len(catNms) == 0
                else [cat for cat in cats if cat["name"] in catNms]
            )
            cats = (
                cats
                if len(catIds) == 0
                else [cat for cat in cats if cat["id"] in catIds]
            )
        ids = [cat["id"] for cat in cats]
        return ids

    def getImgIds(self, imgIds=[], catIds=[]):
        """
        return an integer array of img ids for given ids with all given cats
        """
        ids = set(imgIds)
        for catId in catIds:
            if len(ids) == 0:
                ids = set(self.catToImgs[catId])
            else:
                ids &= set(self.catToImgs[catId])
        return list(ids)

    def loadAnns(self, ids=[]):
        """
        loaded ann objects for integer ids specifying annotations
        """
        if type(ids) == list:
            return [self.anns[id] for id in ids]
        elif type(ids) == int:
            return [self.anns[ids]]

    def loadImgs(self, ids):
        """
        Load loaded img objects with the specified ids.
        """
        return [self.imgs[ids]]

    def getSeg(self, anns):
        """
        get annotations segmentatins
        """
        if len(anns) == 0:
            print("no annotations found")
            return 0

        S = []
        for ann in anns:
            for seg in ann["segmentation"]:
                S.append(seg)
        return S

    def segToMask(self, S, h, w):
        """
        Convert polygon segmentation to binary mask.
          S: polygon segmentation mask, h: target mask height, w: target mask width
        """
        M = np.zeros((h, w))
        for s in S:
            N = len(s)
            rr, cc = polygon(np.array(s[1:N:2]), np.array(s[0:N:2]))  # (y, x)
            M[rr, cc] = 1
        return M



In [ ]:
# Unused
# def coco_to_mask(
#         coco_filename,
#         output_dir,
#         organ,
#         voxelsize_mm=None,
#         output_type="png",
#         show=False,
#         name_prefix="",
# ):
#     """
#     :param coco_filename: coco_filename must include not only name of Coco file, but also it`s full direction (location) in your PC
#     :param output_dir: is used for controll output direction (location) in your PC
#     :param organ: name of segmentation part rom what we want have mask ""
#     :param voxelsize_mm:
#     :param output_type: type of "Save fail" of our program| results]
#     :param show:
#     :return:
#     """
#     # file_path = 'task_cell track 20200226-dii-30las-2pre1-2020_10_26_13_28_36-coco 1.0/annotations/instances_default.json'

#     file_path = coco_filename
#     import datetime

#     t0 = datetime.datetime.now()
#     cv_an = AnnotationTOmask(file_path)
#     t1 = datetime.datetime.now()
#     cv_an1 = cv_an.getImgIds()
#     t2 = datetime.datetime.now()
#     logger.debug(f"{t1 - t0}, {t2 - t1}")
#     logger.debug("getImgIds", cv_an1)
#     # extract the region we want to mask ( Right Kidny,  Liver)

#     # catIds = cv_an.getCatIds(catNms=['cell'])
#     catIds = cv_an.getCatIds(catNms=[organ])
#     # catIds = cv_an.getCatIds(catNms=['Liver'])
#     # catIds = cv_an.getCatIds(catNms=['Left Kidney'])

#     # logger.debug("catIds", catIds)
#     imgIds = cv_an.getImgIds(catIds=catIds)
#     if len(imgIds) == 0:
#         logger.warning(f"Label '{organ}' not found.")
#     # logger.debug(f"imgIds={imgIds}")
#     Path(output_dir).mkdir(parents=True, exist_ok=True)

#     # TODO allow storing into one file
#     #  imgName = cv_an.dataset['images'][0]
#     #  M = np.zeros([imgName['width'], imgName['height'],len(cv_an["images"])])
#     #  ...

#     # create empty files
#     for imgName in cv_an.dataset["images"]:
#         # logger.debug(f"imgName={imgName}")
#         M = np.zeros([imgName["width"], imgName["height"]])
#         image_path = Path(output_dir) / organ / (
#                 name_prefix +
#                 # Path(imgName["file_name"]).name + "." + output_type
#                 imgName["file_name"] + "." + output_type
#         )
#         image_path.parent.mkdir(parents=True, exist_ok=True)
#         # logger.debug(f"image_path={image_path}")
#         skimage.io.imsave(image_path, np.uint8(M), check_contrast=False)
#         # plt.imsave(image_path, np.uint8(M), cmap = 'gray')

#     # rewrite images with label
#     for im in tqdm.tqdm(imgIds):
#         # logger.debug(f"iM{im}")
#         imgName = cv_an.loadImgs(im)[0]
#         anns_ids = cv_an.getAnnIds(imgIds=imgName["id"], catIds=catIds)
#         anns = cv_an.loadAnns(anns_ids)
#         S = cv_an.getSeg(anns)
#         M = cv_an.segToMask(S, imgName["width"], imgName["height"]) * 255
#         if show:
#             plt.figure()
#             plt.imshow(M)
#             plt.show()
#             plt.close()
#         # image_path = Path(output_dir) / ('MaskOfPIG_'+str(imgName['file_name'])+'.'+output_type)
#         image_path = Path(output_dir) / organ / (
#                 name_prefix +
#                 # Path(imgName["file_name"]).name + "." + output_type
#                 imgName["file_name"] + "." + output_type
#         )
#         # logger.debug(image_path)
#         un = np.unique(M)
#         if len(un) < 2:
#             logger.warning(f"No data found in annotation {imgName['file_name']}")
#         skimage.io.imsave(image_path, np.uint8(M), check_contrast=False)
#         plt.imsave(image_path, np.uint8(M), cmap="gray")

#     logger.debug("Done")


In [ ]:
# json_path = Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_annotation\annotations_\pilsen_pigs_coco_from_cvat/instances_default.json")
# # json_path = Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_annotation\annotations_\tx028_coco\annotations\instances_default.json")
# assert json_path.exists()

In [ ]:

# file_path = json_path
# organ="Left Kidney"
# 
# 
# import datetime
# 
# t0 = datetime.datetime.now()
# cv_an = AnnotationTOmask(file_path)
# t1 = datetime.datetime.now()
# cv_an1 = cv_an.getImgIds()
# t2 = datetime.datetime.now()
# logger.debug(f"{t1 - t0}, {t2 - t1}")
# logger.debug("getImgIds", cv_an1)
# # extract the region we want to mask ( Right Kidny,  Liver)
# 
# # catIds = cv_an.getCatIds(catNms=['cell'])
# catIds = cv_an.getCatIds(catNms=[organ])
# # catIds = cv_an.getCatIds(catNms=['Liver'])
# # catIds = cv_an.getCatIds(catNms=['Left Kidney'])
# 
# logger.debug("catIds", catIds)
# imgIds = cv_an.getImgIds(catIds=catIds)
# if len(imgIds) == 0:
#     logger.warning(f"Label '{organ}' not found.")


In [56]:

coco_path= Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_annotation\annotations_\pilsen_pigs_coco_from_cvat/instances_default.json")
# output_path_png= Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_annotation\masks_from_annotations\2024-07_png")
# output_path_mhd = Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_annotation\masks_from_annotations\2024-07_mhd")
output_path_nii = Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_annotation\masks_from_annotations\2024-07_nii")


coco_path= dataset_root / r"transplantation_annotation/annotations_/pilsen_pigs_coco_from_cvat/instances_default.json"
output_path_nii = output_path / r"transplantation_annotation/masks_from_annotations/2024-07_nii"



In [57]:
cv_an = AnnotationTOmask(coco_path)

loading annotations ...
annotations loaded!
creating index...
index created! 17016 images, 6396 annotations, 9 categories


In [58]:
# len(cv_an.dataset["images"])
cv_an.dataset["images"][0]


{'id': 1,
 'width': 512,
 'height': 512,
 'file_name': 'Tx040D_Ven/Tx040D_Vensoubor_01123.jpg',
 'license': 0,
 'flickr_url': '',
 'coco_url': '',
 'date_captured': 0}

In [59]:
cv_an.dataset["categories"]

[{'id': 1, 'name': 'Heart', 'supercategory': ''},
 {'id': 2, 'name': 'Thoracic Cavity', 'supercategory': ''},
 {'id': 3, 'name': 'Left Kidney', 'supercategory': ''},
 {'id': 4, 'name': 'Right Kidney', 'supercategory': ''},
 {'id': 5, 'name': 'Liver', 'supercategory': ''},
 {'id': 6, 'name': 'Portal Vein', 'supercategory': ''},
 {'id': 7, 'name': 'Vena Cava', 'supercategory': ''},
 {'id': 8, 'name': 'Aorta', 'supercategory': ''},
 {'id': 9, 'name': 'Portal Vein Entry Point', 'supercategory': ''}]

In [62]:
dataset_metadata

{'Tx018D_Art': {'filename': 'Tx018D_Art.nii.gz',
  'shape': (512, 512, 284),
  'voxelsize_mm': [0.6525976657867432, 0.6525976657867432, 0.800000011920929]},
 'Tx039D_Art': {'filename': 'Tx039D_Art.nii.gz',
  'shape': (512, 512, 736),
  'voxelsize_mm': [0.8959023356437683, 0.8959023356437683, 0.800000011920929]},
 'Tx047D_Art': {'filename': 'Tx047D_Art.nii.gz',
  'shape': (512, 512, 750),
  'voxelsize_mm': [0.7210000157356262, 0.7210000157356262, 0.800000011920929]},
 'Tx041D_Art': {'filename': 'Tx041D_Art.nii.gz',
  'shape': (512, 512, 641),
  'voxelsize_mm': [0.7883945107460022, 0.7883945107460022, 0.800000011920929]},
 'Tx027D_Art': {'filename': 'Tx027D_Art.nii.gz',
  'shape': (512, 512, 710),
  'voxelsize_mm': [0.695562481880188, 0.695562481880188, 0.800000011920929]},
 'Tx031D_Art': {'filename': 'Tx031D_Art.nii.gz',
  'shape': (512, 512, 659),
  'voxelsize_mm': [0.7245117425918579, 0.7245117425918579, 0.800000011920929]},
 'Tx017D_Ven': {'filename': 'Tx017D_Ven.nii.gz',
  'shape': 

In [60]:

class SaveMasksFromCVAT:
    
    def __init__(self, coco_path, output_path_mhd, dataset_metadata, suffix=".nii.gz"):
        self.coco_path = coco_path
        self.output_path_mhd = output_path_mhd
        self.dataset_metadata = dataset_metadata
        self.suffix = suffix # .mhd, .nii.gz
        cv_an = AnnotationTOmask(coco_path)
        # sort images by file_name
        cv_an.dataset["images"] = sorted(cv_an.dataset["images"], key=lambda x: x["file_name"])

        dataset_metadata_cvat = {}
        for i, img in enumerate(cv_an.dataset["images"]):
            ct_name = Path(img["file_name"]).parent.name
            if ct_name not in dataset_metadata_cvat:
                dataset_metadata_cvat[ct_name] = dict(
                    width=img["width"],
                    height=img["height"],
                    count=1,
                )
            else:
                dataset_metadata_cvat[ct_name]["count"] += 1
                
        self.cv_an = cv_an
        self.dataset_metadata_cvat = dataset_metadata_cvat
    
        # with open(dataset_root / "dataset_metadata_cvat.json", "w") as f:
        #     json.dump(dataset_metadata_cvat, f)
    
    def get_list_of_organs(self):
        return [element["name"] for element in self.cv_an.dataset["categories"]]

    def create_empty_mask_and_name(self, imgId:int):
        dataset_metadata = self.dataset_metadata
        img_record = cv_an.loadImgs(imgId)[0]
        ct_name = Path(img_record["file_name"]).parent.name
        if ct_name in dataset_metadata:
            shape = dataset_metadata[ct_name]["shape"]
            voxelsize_mm = dataset_metadata[ct_name]["voxelsize_mm"]
        else:
            logger.warning(f"CT {ct_name} not found in dataset_metadata, voxelsize set to [1., 1., 1.]")
            # shape = [self.dataset_metadata_cvat[ct_name]["count"], self.dataset_metadata_cvat[ct_name]["width"], self.dataset_metadata_cvat[ct_name]["height"]]
            shape = [self.dataset_metadata_cvat[ct_name]["width"], self.dataset_metadata_cvat[ct_name]["height"], self.dataset_metadata_cvat[ct_name]["count"]]
            voxelsize_mm = [1., 1., 1.]

        mask3d = np.zeros(shape, dtype=np.uint8)


        return mask3d, ct_name, voxelsize_mm
    
    def save_masks_for_organ(self, organ:str):
        cv_an = self.cv_an
        # organ = "Left Kidney"
        catIds = cv_an.getCatIds(catNms=[organ])
        # catIds = cv_an.getCatIds(catNms=['Liver'])
        # catIds = cv_an.getCatIds(catNms=['Left Kidney'])

        # logger.debug("catIds", catIds)
        imgIds = cv_an.getImgIds(catIds=catIds)
        imgIds = sorted(imgIds, key=lambda x: cv_an.loadImgs(x)[0]["file_name"])


        if len(imgIds) == 0:
            logger.warning(f"Label '{organ}' not found.")
            return
        imgId = imgIds[0]
        mask3d, ct_name, voxelsize_mm = self.create_empty_mask_and_name(imgId)
        # n_processed = 0

        for im in tqdm.tqdm(imgIds):
            # logger.debug(f"iM{im}")
            img_record = cv_an.loadImgs(im)[0]
            ct_name_i = Path(img_record["file_name"]).parent.name
            if ct_name_i != ct_name:
                # save mask3d and prepare new one
                jpg_path = self.output_path_mhd / ct_name / (ct_name + "_" + organ.lower().replace(" ", "_") +  ".jpg")
                mhd_path = self.output_path_mhd / ct_name / (ct_name + "_" + organ.lower().replace(" ", "_") +  self.suffix)
                mhd_path.parent.mkdir(parents=True, exist_ok=True)
                io3d.write(mask3d, mhd_path, metadata={"voxelsize_mm": voxelsize_mm})
                # save projection to jpg
                projection = mask3d.astype(float).sum(axis=0)
                if projection.max() > 0:
                    projection = (projection / projection.max())
                    # logger.info(f"{ct_name} {organ} is ok")
                
                projection = (projection * 255.).astype(np.uint8)
                skimage.io.imsave(jpg_path, projection, check_contrast=False)
                mask3d, ct_name, voxelsize_mm = self.create_empty_mask_and_name(im)


            anns_ids = cv_an.getAnnIds(imgIds=img_record["id"], catIds=catIds)
            anns = cv_an.loadAnns(anns_ids)
            S = cv_an.getSeg(anns)
            mask2d = cv_an.segToMask(S, img_record["width"], img_record["height"]) * 255
            mask2d = mask2d.astype(np.uint8)

            filestem = (Path(img_record["file_name"]).stem)
            # get last number from filename. filename is i.e. Tx028D_Vensoubor_00566
            slice_id = int(filestem.split("_")[-1])
            mask3d[:,:, slice_id] = mask2d
            # mask3d[slice_id] = mask2d
            # if np.max(mask2d)> 0:
            #     n_processed += 1

    def save_masks(self, organs=None):
        if organs is None:
            organs = self.get_list_of_organs()
            
        for organ in organs:
            self.save_masks_for_organ(organ)

In [61]:
svmasks = SaveMasksFromCVAT(coco_path, output_path_nii, dataset_metadata, suffix=".nii.gz")
svmasks.save_masks()

loading annotations ...


2024-10-03 11:20:16.411 | WARNING  | __main__:create_empty_mask_and_name:41 - CT Tx028D_Ven not found in dataset_metadata, voxelsize set to [1., 1., 1.]


annotations loaded!
creating index...
index created! 17016 images, 6396 annotations, 9 categories


100%|██████████| 983/983 [00:26<00:00, 37.56it/s]
2024-10-03 11:20:42.606 | WARNING  | __main__:create_empty_mask_and_name:41 - CT Tx028D_Ven not found in dataset_metadata, voxelsize set to [1., 1., 1.]
100%|██████████| 306/306 [00:06<00:00, 44.09it/s]
2024-10-03 11:20:49.570 | WARNING  | __main__:create_empty_mask_and_name:41 - CT Tx028D_Ven not found in dataset_metadata, voxelsize set to [1., 1., 1.]
100%|██████████| 1010/1010 [00:24<00:00, 41.35it/s]
2024-10-03 11:21:14.023 | WARNING  | __main__:create_empty_mask_and_name:41 - CT Tx028D_Ven not found in dataset_metadata, voxelsize set to [1., 1., 1.]
100%|██████████| 1006/1006 [00:24<00:00, 41.09it/s]
2024-10-03 11:21:38.533 | WARNING  | __main__:create_empty_mask_and_name:41 - CT Tx028D_Ven not found in dataset_metadata, voxelsize set to [1., 1., 1.]
100%|██████████| 1673/1673 [00:36<00:00, 46.22it/s] 
2024-10-03 11:22:14.758 | WARNING  | __main__:create_empty_mask_and_name:41 - CT Tx040D_Ven not found in dataset_metadata, voxelsiz